<a href="https://colab.research.google.com/github/Joao-2003/Operational-Systems-/blob/main/EstadosProcessos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

# Enumeração para os estados do processo para maior clareza
class EstadoProcesso:
    PRONTO = "PRONTO"
    EXECUTANDO = "EXECUTANDO"
    BLOQUEADO = "BLOQUEADO"
    TERMINADO = "TERMINADO"

class Processo:
    """
    Representa o Bloco de Controle de Processo (PCB).
    Armazena todos os atributos e o estado de um único processo.
    """
    def __init__(self, pid, tempo_total_execucao):
        self.pid = pid
        self.tempo_total_execucao = tempo_total_execucao
        self.tp = 0  # Tempo de Processamento (total de ciclos executados)
        self.cp = 1  # Contador de Programa (TP + 1)
        self.ep = EstadoProcesso.PRONTO  # Estado do Processo
        self.nes = 0 # Número de vezes que realizou E/S
        self.n_cpu = 0 # Número de vezes que usou a CPU

    def __str__(self):
        """Representação em string para fácil visualização."""
        return (f"PID: {self.pid} | TP: {self.tp}/{self.tempo_total_execucao} | "
                f"CP: {self.cp} | Estado: {self.ep} | E/S: {self.nes} | "
                f"N_CPU: {self.n_cpu}")

    def executar_ciclo(self):
        """Simula a execução de um único ciclo de CPU."""
        if self.tp < self.tempo_total_execucao:
            self.tp += 1
            self.cp = self.tp + 1

    def is_terminado(self):
        """Verifica se o processo concluiu sua execução."""
        return self.tp == self.tempo_total_execucao

In [2]:
import time

class Escalonador:
    """
    Implementa a lógica do escalonador Sequencial Estático e o ciclo principal.
    """
    def __init__(self, processos, quantum, prob_es, prob_desbloqueio):
        self.lista_de_processos = processos # Lista estática e ordenada por PID
        self.quantum = quantum
        self.prob_es = prob_es
        self.prob_desbloqueio = prob_desbloqueio
        self.ciclo_global = 0
        self.log_file = "Tabela_de_Processos.txt"

        # Inicializa o arquivo de log com um cabeçalho
        with open(self.log_file, 'w') as f:
            f.write("CicloGlobal,PID,TP,CP,EstadoAnterior,EstadoNovo,NES,N_CPU\n")

    def _log_troca_contexto(self, processo, estado_anterior, estado_novo):
        """
        Registra a troca de contexto no console e no arquivo de log.
        """
        print(f"** Troca de Contexto no ciclo {self.ciclo_global} **")
        print(f"   Processo {processo.pid}: {estado_anterior} >>> {estado_novo}")
        print(f"   Estado salvo: {processo}\n")

        with open(self.log_file, 'a') as f:
            log_entry = (f"{self.ciclo_global},{processo.pid},{processo.tp},{processo.cp},"
                         f"{estado_anterior},{estado_novo},{processo.nes},{processo.n_cpu}\n")
            f.write(log_entry)

    def simular(self):
        """
        Executa o ciclo principal da simulação até que todos os processos terminem.
        """
        print(">>> INÍCIO DA SIMULAÇÃO <<<\n")

        num_processos_terminados = 0
        total_processos = len(self.lista_de_processos)

        while num_processos_terminados < total_processos:
            # Itera sobre a lista de processos em ordem fixa (0 a 9)
            for processo in self.lista_de_processos:

                # 1. Ignora processos já terminados
                if processo.ep == EstadoProcesso.TERMINADO:
                    continue

                # 2. Lida com processos bloqueados
                if processo.ep == EstadoProcesso.BLOQUEADO:
                    if random.random() < self.prob_desbloqueio:
                        processo.ep = EstadoProcesso.PRONTO
                        print(f"--- Evento no ciclo {self.ciclo_global}: Processo {processo.pid} desbloqueado. Agora está PRONTO.\n")
                        self.ciclo_global += 1
                        continue
                    # Se não desbloquear, simplesmente passa para o próximo
                    continue

                # 3. Lida com processos prontos para executar
                if processo.ep == EstadoProcesso.PRONTO:
                    processo.ep = EstadoProcesso.EXECUTANDO
                    processo.n_cpu += 1
                    print(f"--- Ciclo {self.ciclo_global}: Despachando processo {processo.pid} para a CPU (N_CPU: {processo.n_cpu}).\n")

                    interrompido = False
                    for _ in range(self.quantum):
                        self.ciclo_global += 1
                        processo.executar_ciclo()

                        if processo.is_terminado():
                            estado_anterior = processo.ep
                            processo.ep = EstadoProcesso.TERMINADO
                            print("="*50)
                            print(f"PROCESSO {processo.pid} TERMINOU NO CICLO {self.ciclo_global}")
                            print(f"Estado Final: {processo}")
                            print("="*50 + "\n")
                            num_processos_terminados += 1
                            interrompido = True
                            break

                        if random.random() < self.prob_es:
                            estado_anterior = processo.ep
                            processo.ep = EstadoProcesso.BLOQUEADO
                            processo.nes += 1
                            self._log_troca_contexto(processo, estado_anterior, EstadoProcesso.BLOQUEADO)
                            interrompido = True
                            break

                    if not interrompido:
                        estado_anterior = processo.ep
                        processo.ep = EstadoProcesso.PRONTO
                        self._log_troca_contexto(processo, estado_anterior, EstadoProcesso.PRONTO)


        print(">>> FIM DA SIMULAÇÃO: Todos os processos foram concluídos. <<<\n")

In [5]:
# main.py

if __name__ == "__main__":
    # Definição dos tempos de execução para cada PID
    tempos_de_execucao = {
        0: 10000, 1: 5000, 2: 7000, 3: 3000, 4: 3000,
        5: 8000, 6: 2000, 7: 5000, 8: 4000, 9: 10000
    }

    # Criação da lista inicial de processos
    lista_de_processos = []
    for pid, tempo in tempos_de_execucao.items():
        lista_de_processos.append(Processo(pid=pid, tempo_total_execucao=tempo))

    print("Estado inicial dos processos:")
    for p in lista_de_processos:
        print(p)
    print("\n" + "-"*60 + "\n")

    # Parâmetros da simulação
    QUANTUM = 1000
    PROBABILIDADE_ES = 0.01  # 1%
    PROBABILIDADE_DESBLOQUEIO = 0.30 # 30%

    # Instanciação e execução do escalonador
    escalonador = Escalonador(
        processos=lista_de_processos,
        quantum=QUANTUM,
        prob_es=PROBABILIDADE_ES,
        prob_desbloqueio=PROBABILIDADE_DESBLOQUEIO
    )

    escalonador.simular()

Estado inicial dos processos:
PID: 0 | TP: 0/10000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 1 | TP: 0/5000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 2 | TP: 0/7000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 3 | TP: 0/3000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 4 | TP: 0/3000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 5 | TP: 0/8000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 6 | TP: 0/2000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 7 | TP: 0/5000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 8 | TP: 0/4000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0
PID: 9 | TP: 0/10000 | CP: 1 | Estado: PRONTO | E/S: 0 | N_CPU: 0

------------------------------------------------------------

>>> INÍCIO DA SIMULAÇÃO <<<

--- Ciclo 0: Despachando processo 0 para a CPU (N_CPU: 1).

** Troca de Contexto no ciclo 15 **
   Processo 0: EXECUTANDO >>> BLOQUEADO
   Estado salvo: PID: 0 | TP: 15/10000 | CP: 16 | Estado: BLOQUEADO | E/S: 1 | N_CPU: 1

-